In [ ]:
!wget -O 'dataset' 'https://storage.googleapis.com/kaggle-data-sets/111880%2F269359%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1597061976&Signature=Pn%2FcyEOBBGT3%2BNIhGi%2FIGolMNaQoCnk2bjc6MACdpRon9sMrwpsRK%2FO2p2PmUfT3xOWCHxXj5BJVsegwAJRBs1tVWDfELMXtMoB1r%2F%2BurngYK3Ytmcx21v0zkmflWRsjIDqZDGs20yN%2BY1Fwf98viPXis9H7LiXcDbkayL4ZMAFirrnoVVDxp0Eb3vOsjC1Esw1pTBGMXr%2Fh3iDptoy75%2FMCVs4hqbaMKdxrqro6z3NLHB%2FIlKmlLvcv5GOupBExiT45XcSO43tKSneu2dLayZAGBqSXAUQakhT6dYww17MdkMzv%2F2MAAm97FKp%2BhHpY95YMVG%2Ftvh0oghaExmQNaA%3D%3D'

--2020-08-07 14:23:50--  https://storage.googleapis.com/kaggle-data-sets/111880%2F269359%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1597061976&Signature=Pn%2FcyEOBBGT3%2BNIhGi%2FIGolMNaQoCnk2bjc6MACdpRon9sMrwpsRK%2FO2p2PmUfT3xOWCHxXj5BJVsegwAJRBs1tVWDfELMXtMoB1r%2F%2BurngYK3Ytmcx21v0zkmflWRsjIDqZDGs20yN%2BY1Fwf98viPXis9H7LiXcDbkayL4ZMAFirrnoVVDxp0Eb3vOsjC1Esw1pTBGMXr%2Fh3iDptoy75%2FMCVs4hqbaMKdxrqro6z3NLHB%2FIlKmlLvcv5GOupBExiT45XcSO43tKSneu2dLayZAGBqSXAUQakhT6dYww17MdkMzv%2F2MAAm97FKp%2BhHpY95YMVG%2Ftvh0oghaExmQNaA%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.187.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.187.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 363152213 (346M) [application/zip]
Saving to: ‘dataset’

dataset             100%[===================>] 346.33M  21.9MB/s    in 17s     

2020-08-07 14:

In [ ]:
src_zip = 'dataset'
cpy_zip = 'dataset_folder/' 

import zipfile
with zipfile.ZipFile(src_zip, 'r') as zip_ref:
    zip_ref.extractall(cpy_zip)


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import random

from keras.utils.np_utils import to_categorical


In [ ]:
def make_lab_dict(path_dataset):
    folders = os.listdir(path_dataset)
    folders = np.sort(folders)  
    labs = list(range(0,len(folders)))
    lab_dict = dict(zip(folders, labs))
    
    return lab_dict

def get_paths(path_dataset):
    all_paths = glob(path_dataset+'/'+'*'+'/'+'*')
    return all_paths

In [ ]:
def my_datagen(path_dataset, lab_dict, batch_size=64, out_shape=(150,150,3)):
    
    while True: 
        all_paths = get_paths(path_dataset)
        selected_paths = random.sample(all_paths, batch_size)
        
        batch_images = []
        batch_labels = []

        r_h = random.randint(20,40)
        r_w = random.randint(20,40)
        for i in range(0,len(selected_paths)):
            img = cv2.imread(selected_paths[i])
            img = cv2.resize(img,(out_shape[0]+r_h,out_shape[1]+r_w))

            img_category = selected_paths[i].split('/')[-2]
            lab = lab_dict[img_category]
            lab = to_categorical(lab, num_classes=6)
            # print(img_category)
            # print(lab)

            # img = tf.convert_to_tensor(img, dtype=None, dtype_hint=None, name=None)
            # lab = tf.convert_to_tensor(lab, dtype=None, dtype_hint=None, name=None)



            batch_images.append(img)
            batch_labels.append(lab)
        
        yield np.array(batch_images)/255, np.array(batch_labels)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

from tensorflow.keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import tensorflow as tf

In [ ]:
def build_model():
    model = VGG16(weights='imagenet', include_top=False)
    x = model.output

    x = Conv2D(1024, padding='valid',kernel_size=[1,1], activation='relu')(x)
    x = Conv2D(1024, padding='valid',kernel_size=[1,1], activation='relu')(x)
    x = Conv2D(6, padding='valid',kernel_size=[1,1], activation='softmax')(x)
    x = GlobalMaxPooling2D()(x)

    new_model =  Model(model.input, x)

    return new_model


In [ ]:
def init_model(model_name='my_model', path_ckpt='checkpoint/'):
    complete_path = path_ckpt+model_name+'.h5'

    model = build_model()
       
    if not os.path.isdir(path_ckpt):
        os.mkdir(path_ckpt)

    names = os.listdir(path_ckpt)
    if complete_path.split('/')[-1] in names:
        print('Loading checkpoint...')
        model.load_weights(complete_path)
    else:
        print('Training from scratch')

    return model, complete_path

    

In [ ]:
def train_model(path_dataset, batch_size=64, epochs=2):
    model, checkpoint_path = init_model()
    
    checkpoint=ModelCheckpoint(filepath=checkpoint_path,monitor='loss',save_best_only=True,
                           save_weights_only=False,verbose=0,mode="auto")
    
    # checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='loss', save_best_only=False, 
    #                              save_weights_only=False, mode='auto', period=2)

    callbacks=[checkpoint]
    print('learning rate = ',learn_rate)
    opt = tf.keras.optimizers.Adam(lr = learn_rate)
    
    model.compile(optimizer=opt,loss="categorical_crossentropy")

    lab_dict = make_lab_dict(path_dataset)
    print(lab_dict)
    print(model.summary())
    data_size = len(get_paths(path_dataset))

    train_gen = my_datagen(path_dataset, lab_dict, batch_size=batch_size)
    model.fit_generator(train_gen, epochs=epochs, callbacks = callbacks,steps_per_epoch=int(data_size/batch_size))

    # save trained model
    #model.save('checkpoint/' + 'trained_model.h5')


In [ ]:
global learn_rate
learn_rate = 0.0001

batch_size = 256
#input_shape = (150,150,3)
path_dataset = 'dataset_folder/seg_train/seg_train/'

In [ ]:
train_model(path_dataset, batch_size, 300)


58892288/58889256 [==============================] - 2s 0us/step
Training from scratch
learning rate =  0.0001
{'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
___________________________________________________________

ResourceExhaustedError: ignored